In [ ]:

# !pip install langdetect
!pip install wandb -qU
# !pip install fasttext
!pip install h5py
!pip install gensim

# Standard library imports
# from langdetect import detect, detect_langs
import os
import warnings
import h5py
import gc
from torch.cuda.amp import autocast, GradScaler

# Data processing and numerical libraries
import numpy as np
import pandas as pd
# import scipy.sparse as sp
# import dask.dataframe as dd
from sklearn.model_selection import train_test_split

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and recommendation libraries
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import wandb
import torch

from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import time

# import fasttext
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
from transformers import BertPreTrainedModel
from transformers import BertModel, BertTokenizer
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.model_selection import train_test_split
# from pyspark.ml.feature import StringIndexer, VectorAssembler
# from pyspark.ml.recommendation import ALS
# from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# from pyspark.ml.evaluation import RegressionEvaluator
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, udf
# from pyspark.sql.types import StringType
# from pyspark.sql.types import IntegerType, FloatType, StringType
# from pyspark.ml import Pipeline

# MLflow for experiment tracking
# import mlflow
# from pyngrok import ngrok

# IPython for displaying outputs
from IPython.display import display


# Suppress warnings
warnings.filterwarnings('ignore')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Convert df to structured array
# RECAP: sturctured arrays are more effieicnt cause they store information in continguous memory

# yelp_df = pd.read_parquet('/content/drive/MyDrive/Machine Learning/BRS/final_df.parquet')

# Define the structured array dtype
# dtype = [
#     ('user_num_id', 'i4'),
#     ('business_num_id', 'i4'),
#     ('log_business_review_count_norm', 'f4'),
#     ('bus_avg_rating_norm', 'f4'),
#     ('region_code', 'i4'),
#     ('state_code', 'i4'),
#     ('city_code', 'i4'),
#     ('day_of_year', 'i4'),
#     ('day_of_week', 'i4'),
#     ('mean_centered_rating', 'f4'),
#     ('u_pca1', 'f4'),
#     ('u_pca2', 'f4'),
#     ('u_pca3', 'f4'),
# ]

# # Convert the DataFrame to a structured array
# structured_array = np.array(list(yelp_df.to_records(index=False)), dtype=dtype)

# # Save the structured array as a memory-mapped file
# np.save('/content/drive/MyDrive/Machine Learning/BRS/Numpy Arrays/yelp_structured.npy', structured_array,)


In [ ]:
# import numpy as np
# import torch
# from torch.utils.data import Dataset, DataLoader

# class Lazy_CFDataset(Dataset):
#     def __init__(self, yelp_file_path, categories_file_path, token_file_path, att_file_path):
#         # Load memory-mapped structured array
#         try:
#             # Load the structured array
#             self.yelp_data = np.load(yelp_file_path, mmap_mode='r')  # Open in read-only mode

#             # Load other file paths
#             self.categories_file_path = categories_file_path
#             self.token_file_path = token_file_path
#             self.att_file_path = att_file_path

#             # Pre-compute the dataset length
#             self.length = len(self.yelp_data)

#         except Exception as e:
#             print(f"Error loading data: {e}")

#     def __len__(self):
#         return self.length

#     def __getitem__(self, idx):
#         # Load the tensors when accessing items
#         user_id = torch.tensor(self.yelp_data['user_num_id'][idx], dtype=torch.int32)  # Shape: (N,)
#         business_id = torch.tensor(self.yelp_data['business_num_id'][idx], dtype=torch.int32)  # Shape: (N,)
#         region_id = torch.tensor(self.yelp_data['region_code'][idx], dtype=torch.int32)  # Shape: (N,)
#         dotw = torch.tensor(self.yelp_data['day_of_week'][idx], dtype=torch.int32)  # Shape: (N,)
#         doty = torch.tensor(self.yelp_data['day_of_year'][idx], dtype=torch.int32)  # Shape: (N,)
#         mean_centered_rating = torch.tensor(self.yelp_data['mean_centered_rating'][idx], dtype=torch.float32)  # Shape: (N,)

#         # Group PCA features into a single tensor of shape (3,)
#         pca_features = torch.tensor([
#             self.yelp_data['u_pca1'][idx],
#             self.yelp_data['u_pca2'][idx],
#             self.yelp_data['u_pca3'][idx]
#         ], dtype=torch.float32)  # Shape: (3,)

#         # Load other tensors
#         tokens = torch.load(self.token_file_path)[idx]  # Shape: (Token_Length,)
#         att_mask = torch.load(self.att_file_path)[idx]  # Shape: (Token_Length,)
#         category = torch.load(self.categories_file_path)[idx]  # Shape: (N,)

#         return {
#             'user_id': user_id,
#             'business_id': business_id,
#             'region_id': region_id,
#             'dotw': dotw,
#             'doty': doty,
#             'pca_features': pca_features,  # (3,)
#             'tokens': tokens,
#             'att_mask': att_mask,
#             'category': category
#         }


Batch loaded successfully


In [ ]:
class CFDataset(Dataset):
    def __init__(self, yelp_file_path, categories_file_path, token_file_path, mem_map=False):
        # Load memory-mapped structured array
        try:
            # Load the structured array
            if mem_map:
              yelp_data = np.load(yelp_file_path, mmap_mode='r')
            else:
              yelp_data = np.load(yelp_file_path)

            # Extract features from the structured array
            self.user_id = torch.tensor(yelp_data['user_num_id'], dtype=torch.int32)  # Shape: (N,)
            self.business_id = torch.tensor(yelp_data['business_num_id'], dtype=torch.int32)  # Shape: (N,)
            self.region_id = torch.tensor(yelp_data['region_code'], dtype=torch.int32)  # Shape: (N,)
            self.dotw = torch.tensor(yelp_data['day_of_week'], dtype=torch.int32)  # Shape: (N,)
            self.doty = torch.tensor(yelp_data['day_of_year'], dtype=torch.int32)  # Shape: (N,)
            self.mean_centered_rating = torch.tensor(yelp_data['mean_centered_rating'], dtype=torch.float32)  # Shape: (N,)

            # # Group PCA features into a single tensor of shape (N, 3)
            self.pca_features = torch.stack([
                torch.tensor(yelp_data['u_pca1'], dtype=torch.float32),
                torch.tensor(yelp_data['u_pca2'], dtype=torch.float32),
                torch.tensor(yelp_data['u_pca3'], dtype=torch.float32)
            ], dim=1)  # Shape: (N, 3)

            # Load other memory-mapped tensors
            self.token_emb = torch.load(token_file_path)  # Shape: (N, Token_Length)
            self.categories = torch.load(categories_file_path)  # Shape: (N,)

            # Pre-compute the dataset length based on tokens
            self.length = len(self.token_emb)

        except Exception as e:
            print(f"Error loading data: {e}")

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        # Get a batch of data
        user_id = self.user_id[idx]
        business_id = self.business_id[idx]
        region_id = self.region_id[idx]
        dotw = self.dotw[idx]
        doty = self.doty[idx]
        pca_features = self.pca_features[idx]  # (3,)
        token_emb = self.token_emb[idx]
        category = self.categories[idx].to(torch.long)

        return {
            'user_id': user_id,
            'business_id': business_id,
            'region_id': region_id,
            'dotw': dotw,
            'doty': doty,
            'pca_features': pca_features,  # (3,)
            'tokens': token_emb,
            'category': category
        }, self.mean_centered_rating[idx]


In [ ]:


def gather_and_load_data(rem_yelp_file_path, rem_cat_file_path, rem_token_file_path, save_dir):
    start_time = time.time()

    # Load structured array and tensors from drive
    yelp_array = np.load(rem_yelp_file_path, allow_pickle=True)  # Allow_pickle is needed for structured arrays!!
    token_emb = torch.load(rem_token_file_path)
    categories = torch.load(rem_cat_file_path)

    curr_time = time.time()
    print(f"Data has been loaded from Google Drive")
    print(f"Read time from Google Drive: {curr_time - start_time:.4f} seconds")

    # Prepare the data for splitting
    num_samples = yelp_array.shape[0] 
    indices = np.arange(num_samples)

    # Split data 
    train_indices, temp_indices = train_test_split(indices, test_size=0.2, random_state=42)  
    valid_indices, test_indices = train_test_split(temp_indices, test_size=0.5, random_state=42) 

    # Create datasets 
    train_yelp_data = yelp_array[train_indices]
    valid_yelp_data = yelp_array[valid_indices]
    test_yelp_data = yelp_array[test_indices]

    train_token_emb = token_emb[train_indices]
    valid_token_emb = token_emb[valid_indices]
    test_token_emb = token_emb[test_indices]


    train_categories = categories[train_indices]
    valid_categories = categories[valid_indices]
    test_categories = categories[test_indices]

    # Save the split datasets remotely
    np.save(f'{save_dir}/train_yelp.npy', train_yelp_data)
    np.save(f'{save_dir}/valid_yelp.npy', valid_yelp_data)
    np.save(f'{save_dir}/test_yelp.npy', test_yelp_data)


    torch.save(train_token_emb, f'{save_dir}/train_tok_emb.pt')
    torch.save(valid_token_emb, f'{save_dir}/valid_tok_emb.pt')
    torch.save(test_token_emb, f'{save_dir}/test_tok_emb.pt')

    torch.save(train_categories, f'{save_dir}/train_categories.pt')
    torch.save(valid_categories, f'{save_dir}/valid_categories.pt')
    torch.save(test_categories, f'{save_dir}/test_categories.pt')

    end_time = time.time()
    print(f"Data has been written to Google Drive")
    print(f"Write time from Google Drive: {end_time - curr_time:.4f} seconds")

    # Remove the original versions from memory
    del yelp_array, token_emb, categories

    # Run garbage collection to free up memory
    gc.collect()

    print("Dataset has been split into training, validation, and test sets and saved remotely.")



def create_datasets_from_saved_files(split_dir, dataset_save_dir):

    # Ensure the dataset save directory exists
    os.makedirs(dataset_save_dir, exist_ok=True)

    # Load the saved data files
    train_yelp_data_path = f'{split_dir}/train_yelp.npy'
    valid_yelp_data_path = f'{split_dir}/valid_yelp.npy'
    test_yelp_data_path = f'{split_dir}/test_yelp.npy'

    train_token_emb_path = f'{split_dir}/train_tok_emb.pt'
    valid_token_emb_path = f'{split_dir}/valid_tok_emb.pt'
    test_token_emb_path = f'{split_dir}/test_tok_emb.pt'

    train_categories_path = f'{split_dir}/train_categories.pt'
    valid_categories_path = f'{split_dir}/valid_categories.pt'
    test_categories_path = f'{split_dir}/test_categories.pt'


    # Create Dataset objects
    train_dataset = CFDataset(train_yelp_data_path, train_categories_path, train_token_emb_path)
    valid_dataset = CFDataset(valid_yelp_data_path, valid_categories_path, valid_token_emb_path)
    test_dataset = CFDataset(test_yelp_data_path, test_categories_path, test_token_emb_path)

    # Save the Dataset objects
    torch.save(train_dataset, f'{dataset_save_dir}/train_dataset.pt')
    torch.save(valid_dataset, f'{dataset_save_dir}/valid_dataset.pt')
    torch.save(test_dataset, f'{dataset_save_dir}/test_dataset.pt')

    print("Datasets have been created and saved to:", dataset_save_dir)



def load_datasets(dataset_dir, load_test=False):

    datasets = {}

    # Load training and validation datasets
    try:
        datasets['train'] = torch.load(os.path.join(dataset_dir, 'train_dataset.pt'))
        datasets['valid'] = torch.load(os.path.join(dataset_dir, 'valid_dataset.pt'))
        print("Training and validation datasets loaded successfully.")

        if load_test:
            datasets['test'] = torch.load(os.path.join(dataset_dir, 'test_dataset.pt'))
            print("Test dataset loaded successfully.")

    except FileNotFoundError as e:
        print(f"Error loading datasets: {e}")

    return datasets


# Local Raw Data file paths
yelp_local_path = '/content/yelp_local.npy'
train_categories_path = '/content/train_categories.pt'
train_tok_emb_path = '/content/drive/MyDrive/Machine Learning/BRS/embeddings_tensor.pt'
cf_dataset_path = '/content/cf_dataset.pt'

# Remote Raw Data file paths
rem_yelp_file_path = '/content/drive/MyDrive/Machine Learning/BRS/Numpy Arrays/yelp_structured.npy'
rem_token_file_path = '/content/drive/MyDrive/Machine Learning/BRS/embeddings_tensor.pt'
rem_cat_file_path = '/content/drive/MyDrive/Machine Learning/BRS/categories_tensor.pt'

# Split tensor directory
split_dir = '/content/drive/MyDrive/Machine Learning/BRS/Split Data'

# Final dataset directory, constructed CF datasets
final_datesets_dir = '/content/drive/MyDrive/Machine Learning/BRS/Final Datasets'

# Create Dataset and save to directory


# gather_and_load_data(rem_yelp_file_path, rem_cat_file_path, rem_token_file_path, split_dir)

# create_datasets_from_saved_files(split_dir, final_datesets_dir)


# # # Load datasets
datasets = load_datasets(final_datesets_dir)  # Load only train and valid datasets

# # # Unpack the datasets
train_dataset = datasets['train']
val_dataset = datasets['valid']

Training and validation datasets loaded successfully.


In [ ]:
train_dataset[0][0]['tokens'].shape

torch.Size([768])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HybridCFModel(nn.Module):
    def __init__(self, vocab_size=30522, rank=32, num_users=287116, num_bus=148523, num_regions=11,
                 num_dotw=7, num_doty=367, num_pca=3, category_size=1311, token_length=768, num_heads=4):
        super().__init__()
        self.rank = rank
        self.token_length = token_length

        # Ensures rank is divisible by num_heads
        self.num_heads = num_heads
        self.attention_dim = (rank // num_heads) * num_heads

        # Embeddings categorical features
        self.user_emb = nn.Embedding(num_users, rank)
        self.bus_emb = nn.Embedding(num_bus, rank)
        self.region_emb = nn.Embedding(num_regions, rank)
        self.dotw_emb = nn.Embedding(num_dotw, rank)
        self.doty_emb = nn.Embedding(num_doty, rank)
        self.categories = nn.EmbeddingBag(category_size, rank, mode='mean', padding_idx=0)

        # Layer for numerical PCA features - transforms to rank dimension
        self.pca_layer = nn.Sequential(
            nn.Linear(num_pca, rank),
            nn.ReLU(),
            nn.LayerNorm(rank)
        )

        # Token embedding layer
        self.token_layer = nn.Sequential(
            nn.Linear(token_length, rank),
            nn.ReLU(),
            nn.LayerNorm(rank)
        )

        # Projection layers for multi-head attention
        self.user_proj = nn.Linear(rank, self.attention_dim)
        self.bus_proj = nn.Linear(rank, self.attention_dim)

        self.multihead_attn = nn.MultiheadAttention(embed_dim=self.attention_dim, num_heads=num_heads)

        total_input_size = rank * 8 + self.attention_dim

        # Final layers for the combined CF and CBF features
        self.fc_layers = nn.Sequential(
            nn.Linear(total_input_size, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 1)
        )

    def forward(self, **kwargs):
        # CF features
        user_emb = self.user_emb(kwargs['user_id'])  
        bus_emb = self.bus_emb(kwargs['business_id'])  
        region_emb = self.region_emb(kwargs['region_id']) 
        dotw_emb = self.dotw_emb(kwargs['dotw'])  
        doty_emb = self.doty_emb(kwargs['doty']) 
        cat_emb = self.categories(kwargs['category']) 

        # Content-based filtering features
        pca_features = self.pca_layer(kwargs['pca_features']) 
        tokens_emb = self.token_layer(kwargs['tokens'])

        # Project embeddings to attention dimension
        user_proj = self.user_proj(user_emb)
        bus_proj = self.bus_proj(bus_emb)

        # Multi-head attention for user-business interaction
        user_bus_interaction, _ = self.multihead_attn(user_proj.unsqueeze(0), bus_proj.unsqueeze(0), bus_proj.unsqueeze(0))
        user_bus_interaction = user_bus_interaction.squeeze(0)

        # Combine CF and CBF features
        combined_features = torch.cat([
            user_emb, bus_emb, user_bus_interaction, region_emb, dotw_emb, doty_emb, cat_emb, pca_features, tokens_emb
        ], dim=1)

        # Final prediction
        result = self.fc_layers(combined_features)

        return result.squeeze()

In [ ]:

# # Hyperparameters
rank = 16


# Model / Optimizer Information
batch_size = 2000
lr = .001
num_epochs = 15
betas = (0.9, 0.999)
eps=1e-08
weight_decay=.01 #L2 normalization


# Load datasets
# Example with optimizations
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=6,         # Adjust based on your system
    pin_memory=True        # Accelerates transfer to GPU
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    num_workers=6,         # Adjust based on your system
    pin_memory=True        # Accelerates transfer to GPU
)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

# init device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device : {device}")
# Initialize Model
model = HybridCFModel(rank=rank)
model.to(device)



Device : cuda


HybridCFModel(
  (user_emb): Embedding(287116, 10)
  (bus_emb): Embedding(148523, 10)
  (region_emb): Embedding(11, 10)
  (dotw_emb): Embedding(7, 10)
  (doty_emb): Embedding(367, 10)
  (categories): EmbeddingBag(1311, 10, mode='mean', padding_idx=0)
  (pca_layer): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): ReLU()
    (2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  )
  (token_layer): Sequential(
    (0): Linear(in_features=768, out_features=10, bias=True)
    (1): ReLU()
    (2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  )
  (user_proj): Linear(in_features=10, out_features=8, bias=True)
  (bus_proj): Linear(in_features=10, out_features=8, bias=True)
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=88, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=T

In [ ]:
# # initialize model and define optimization

criterion = nn.MSELoss()


optimizer = torch.optim.AdamW(
    model.parameters(),     # Parameters of the model to optimize
    lr=lr,                 # Learning rate (default is 0.001)
    betas=betas,           # Coefficients for computing running averages of gradient and its square
    eps=eps,               # Term added to the denominator to improve numerical stability
    weight_decay=weight_decay  # Weight decay (L2 penalty) applied directly
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=1)



In [ ]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    elif isinstance(data, dict):
        return {key: to_device(val, device) for key, val in data.items()}
    return data.to(device)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# watch -n 1 nvidia-smi

# Initialize W&B to watch the model
wandb.init(project="Updated Model with Bert Tokenization")
wandb.watch(model, log="all")



# Log model architecture using wandb.config
wandb.config.update({
    "descr": "Even better model?",
    "learning_rate": lr,
    "architecture": "CF",
    "dataset": "Yelp",
    "betas": betas,
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "rank": rank,
    "weight_decay": weight_decay,
    "model_architecture": print(model)
})




cuda


HybridCFModel(
  (user_emb): Embedding(287116, 10)
  (bus_emb): Embedding(148523, 10)
  (region_emb): Embedding(11, 10)
  (dotw_emb): Embedding(7, 10)
  (doty_emb): Embedding(367, 10)
  (categories): EmbeddingBag(1311, 10, mode='mean', padding_idx=0)
  (pca_layer): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): ReLU()
    (2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  )
  (token_layer): Sequential(
    (0): Linear(in_features=768, out_features=10, bias=True)
    (1): ReLU()
    (2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  )
  (user_proj): Linear(in_features=10, out_features=8, bias=True)
  (bus_proj): Linear(in_features=10, out_features=8, bias=True)
  (multihead_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=88, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=T

In [ ]:
start_time = time.time()
for epoch in range(num_epochs):
    model.train()

    for num_iter, (features, target) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch")):
        features = to_device(features, device)
        target = to_device(target, device)

        optimizer.zero_grad()

        # Forward pass (no mixed precision)!!!
        pred = model(**features)
        loss = criterion(pred, target)

        # Backward pass
        loss.backward()
        optimizer.step()

        wandb.log({
            'epoch': epoch,
            'iteration': num_iter,
            'train_loss': loss.item(),
        })

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_features, val_target in val_loader:
            val_features = to_device(val_features, device)
            val_target = to_device(val_target, device)

            # Standard forward pass (no mixed precision)
            val_pred = model(**val_features)
            val_loss += criterion(val_pred, val_target).item()

    val_loss /= len(val_loader)


    scheduler.step(val_loss) 
    wandb.log({'val_loss': val_loss})

    print(f"Epoch: {epoch+1}, Iter: {num_iter}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")

# Document total training time and save model
end_time = time.time()
duration = end_time - start_time

wandb.log({'training_duration': duration})

# Save the model
model_path = "model.pth"
torch.save(model.state_dict(), model_path)

# Log the model artifact to W&B
artifact = wandb.Artifact('model', type='model')
artifact.add_file(model_path)
wandb.log_artifact(artifact)

# Finish W&B run
wandb.finish()

# Print the total training time
print(f"Total training time: {duration / 60:.2f} minutes")


Epoch 1/15: 100%|██████████| 1716/1716 [00:55<00:00, 30.70batch/s]


Epoch: 1, Iter: 1715, Train Loss: 0.0400, Val Loss: 0.5308


Epoch 2/15: 100%|██████████| 1716/1716 [00:56<00:00, 30.53batch/s]


Epoch: 2, Iter: 1715, Train Loss: 0.4912, Val Loss: 0.6325


Epoch 3/15: 100%|██████████| 1716/1716 [00:55<00:00, 31.13batch/s]


...

Epoch: 12, Iter: 1715, Train Loss: 0.1448, Val Loss: 0.4779


Epoch 13/15: 100%|██████████| 1716/1716 [00:55<00:00, 30.88batch/s]


Epoch: 13, Iter: 1715, Train Loss: 0.1075, Val Loss: 0.4779


Epoch 14/15:  72%|███████▏  | 1227/1716 [00:39<00:15, 30.71batch/s]


KeyboardInterrupt: 

In [ ]:
test = train_dataset[3]

In [ ]:
tokens = test[0]['tokens']
mask = test[0]['att_mask']

In [ ]:
tokens = torch.load('/content/drive/MyDrive/Machine Learning/BRS/input_ids.pt')
att = torch.load('/content/drive/MyDrive/Machine Learning/BRS/mask.pt')

In [ ]:
import torch
from transformers import DistilBertModel
from tqdm import tqdm

class DistilBertEmb(nn.Module):
    def __init__(self):
        super().__init__()
        self.distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
        # Freeze parameters, do not want to fine-tune DistilBERT...
        for param in self.distilbert_model.parameters():
            param.requires_grad = False

    def forward(self, tokens, att_mask):
        # Forward pass
        outputs = self.distilbert_model(tokens, attention_mask=att_mask)
        # Get embeddings
        emb = outputs.last_hidden_state
        # Mean Pooling
        pooled_tokens_emb = emb.mean(dim=1)  # Across sequence dimension
        return pooled_tokens_emb


def compute_embeddings(token_tensor, att_mask_tensor, distilbert_model, batch_size=64):  # Reduced batch size

    distilbert_model.eval()

    # Store embeddings
    all_embeddings = []

    # Move tensors to gpu
    token_tensor = token_tensor.to('cuda')
    att_mask_tensor = att_mask_tensor.to('cuda')

    # Loop + progress bar
    for i in tqdm(range(0, token_tensor.size(0), batch_size)):
        # Clear CUDA cache to prevent fragmentation issue
        torch.cuda.empty_cache()

        # Get the current batch
        batch_tokens = token_tensor[i:i + batch_size]
        batch_att_mask = att_mask_tensor[i:i + batch_size]

        # Compute embeddings using mixed precision
        # Cut foward pass runtime in half
        with torch.no_grad():  # Disable gradients for faster computation
            with torch.cuda.amp.autocast():  
                embeddings = distilbert_model(batch_tokens, batch_att_mask)

        # Append the embeddings to the lis
        all_embeddings.append(embeddings.cpu())  

    # Concatenate and return result 
    embeddings_tensor = torch.cat(all_embeddings, dim=0)

    return embeddings_tensor


distilbert_model = DistilBertEmb().to('cuda')  # Move to GPU if available
embeddings_tensor = compute_embeddings(tokens, att, distilbert_model)
torch.save(embeddings_tensor, '/content/drive/MyDrive/Machine Learning/BRS/embeddings_tensor.pt')


100%|██████████| 66993/66993 [3:34:42<00:00,  5.20it/s]


In [ ]:
embeddings_tensor

tensor([[ 0.1058,  0.0931,  0.2348,  ..., -0.0107,  0.0529, -0.1374],
        [-0.0279,  0.0524,  0.2643,  ..., -0.0398,  0.1024, -0.0317],
        [-0.0845,  0.0803,  0.2730,  ...,  0.0253,  0.1690, -0.0934],
        ...,
        [-0.0053,  0.2766,  0.1076,  ..., -0.0313,  0.1434, -0.1631],
        [-0.2171,  0.1897,  0.2255,  ..., -0.1316,  0.2117, -0.1333],
        [ 0.0913,  0.1682,  0.2498,  ...,  0.0447,  0.1635,  0.0219]])

In [ ]:
# torch.save(embeddings_tensor, '/content/drive/MyDrive/Machine Learning/BRS/embeddings_tensor2.pt')


In [ ]:
embeddings_tensor.dtype

torch.float32